# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [3]:
import requests
import pandas as pd
import os 

In [ ]:
bikes_data = pd.read_csv('city_bikes_portland.csv', index_col=0)
bikes_data

url = "https://api.foursquare.com/v3/places/search"
api_key = os.environ["APIfoursquare"]
headers = {"Accept": "application/json","Authorization": api_key}
response_dict = {}
for i in bikes_data.index:
    row = bikes_data.loc[i]
    lat_long = str(row['latitude']) + ',' + str(row['longitude'])
    params = {'radius':1000,'ll':lat_long}
    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    response_dict[lat_long] = data

In [ ]:
response_dict

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
name_list = []
main_category_list = []
location_list = []
distance_list = []
lat_long_list = []
error_list_indexes = []
index_count = 0
for i in response_dict:
    for x in response_dict[i]['results']:
        lat_long_list.append(i)
        name_list.append(x['name'])
        distance_list.append(x['distance'])
        #there are 5 businesses with no category, so when that is the case it is replaced with 'n/a'
        try:
            main_category_list.append(x['categories'][0]['name'])
        except:
            main_category_list.append('n/a')
            error_list_indexes.append(index_count)
        index_count += 1
        #there are 31 businesses with no pirmary address, so when that is the case it is replaced with the formatted_address
        #since that will still give some information on the location even without the primary address
        try:
            location_list.append(x['location']['address'])
        except:
            location_list.append(x['location']['formatted_address'])
            error_list_indexes.append(index_count)
        index_count += 1

df_dict_fs = {'bike_station_location':lat_long_list, 'name':name_list, 'main_category':main_category_list, 'address':location_list, 'distance_away':distance_list}

#print(len(location_list), len(name_list), len(lat_long_list), len(main_category_list), len(distance_list))

Put your parsed results into a DataFrame

In [ ]:
df_fs = pd.DataFrame(df_dict_fs)
df_fs
#exporting to csv file for future use
#df_fs.to_csv('foursquare.csv', index=False)

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [4]:
import requests
import pandas as pd
import os

In [ ]:
bikes_data = pd.read_csv('city_bikes_portland.csv', index_col=0)
bikes_data

url = "https://api.yelp.com/v3/businesses/search"
api_key = os.environ["YELP_API"]
headers = {"accept": "application/json",'Authorization': 'Bearer %s' % api_key}
response_dict_yelp = {}
for i in bikes_data.index:
    row = bikes_data.loc[i]
    lat = row['latitude']
    long = row['longitude']
    lat_long = str(row['latitude']) + ',' + str(row['longitude'])
    params = {'radius':1000,'latitude':lat,'longitude':long }
    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    response_dict_yelp[lat_long] = data

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
name_list = []
main_category_list = []
location_list = []
distance_list = []
lat_long_list = []
rating_list = []
review_count_list = []
status_list = []
price_list = []
error_list_indexes = []
index_count = 0
for i in response_dict_yelp:
    for x in response_dict_yelp[i]['businesses']:
        lat_long_list.append(i)
        name_list.append(x['name'])
        main_category_list.append(x['categories'][0]['title'])
        location_list.append(x['location']['display_address'][0])
        distance_list.append(x['distance'])
        rating_list.append(x['rating'])
        review_count_list.append(x['review_count'])
        if x['is_closed'] == False:
            status_list.append('Open')
        else:
            status_list.append('Closed')
        #there are 530 businesses without a price description, so those were replaced with 'n/a'
        try:
            price_list.append(x['price'])
        except:
            price_list.append('n/a')
            error_list_indexes.append(index_count)
        index_count += 1

df_dict_yelp = {'bike_station_location':lat_long_list, 'name':name_list, 'main_category':main_category_list, 'address':location_list, 
                'price':price_list, 'rating':rating_list, 'review_count':review_count_list, 'status':status_list, 'distance_away':distance_list}
#len(error_list_indexes)

Put your parsed results into a DataFrame

In [ ]:
df_yelp = pd.DataFrame(df_dict_yelp)
df_yelp
#exporting to csv file for future use
#df_yelp.to_csv('yelp.csv', index=False)

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

The data from the Yelp API was notably more comprehensive, providing an extensive business details—particularly pertinent ones such as ratings, review counts, and pricing. Moreover, the Yelp API offered a substantially larger number of businesses in comparison to the FourSquare API. This disparity might stem from the Yelp API's capacity to dynamically adjust the search radius based on the area's density, or possibly from Yelp hosting a broader array of business data. Furthermore, the Yelp API provided precise distance measurements, while the FourSquare API presented distances rounded to the nearest value.

Get the top 10 restaurants according to their rating

In [ ]:
df_yelp.sort_values('rating', ascending=False).head(10)